# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Нужно построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install --upgrade pip --user
!pip install torch
!pip install transformers
!pip install tensorflow --user
!pip install tokenization

Defaulting to user installation because normal site-packages is not writeable
2021-09-26 14:23:23.653888: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2021-09-26 14:23:23.654102: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2021-09-26 14:23:23.654132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.6 MB 1.3 MB/s eta 0:00:01
✔ Download and installation successful
You can now load 

In [2]:
import pandas as pd
import numpy as np
import pymystem3
import nltk
import torch
import tokenization
import transformers
import spacy
import tensorflow
import sys
import codecs
import re


import time
pd.set_option('display.precision',3)

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

from lightgbm import LGBMClassifier

from nltk.corpus import stopwords
from nltk.corpus import stopwords as nltk_stopwords

from pymystem3 import Mystem
from tqdm import notebook

In [4]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
display(df['toxic'].value_counts())
class_ratio = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]
class_ratio

0    143346
1     16225
Name: toxic, dtype: int64

8.834884437596301

Видим, что классы несбалансированны. Продолжим обучать модель без балансировки и посмотрим, будет ли удовлетворять результат поставленной задаче.

In [7]:
df.isna().sum()

text     0
toxic    0
dtype: int64

Очистим данные и проведем лемматизацию текста.

In [8]:
m = Mystem()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = "".join(m.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

df['lemm_text'] = df['text'].apply(lemmatize_text)

df = df.drop(['text'], axis=1)
del m

In [9]:
df.head()

,toxic,lemm_text
0,0,explanation why the edits made under my userna...
1,0,d aww he matches this background colour i m se...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestions on impr...
4,0,you sir are my hero any chance you remember wh...


Разобьем данные на выборки и сохраним тестово-валидационную выборку для предтестового обучения модели.

In [10]:
train_val = df.sample(frac=0.8, random_state=42).copy()
test = df[~df.index.isin(train_val.index)].copy()

In [11]:
train = train_val.sample(frac=0.75, random_state=42).copy()
val = train_val[~train_val.index.isin(train.index)].copy()

In [12]:
len(df) == len(train) + len(val) + len(test)

True

In [13]:
len(train_val) == len(train) + len(val)

True

## Обучение

Создадим счетчик слов с помощью TF-IDF.

In [14]:
vect = TfidfVectorizer(stop_words='english')
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [15]:
train_val_X = vect.fit_transform(train_val['lemm_text'])
train_val_y = train_val['toxic']
test_X = vect.transform(test['lemm_text'])
test_y = test['toxic']

In [16]:
X = vect.fit_transform(train['lemm_text'])
y = train['toxic']
val_X = vect.transform(val['lemm_text'])
val_y = val['toxic']

**LogisticRegression**

In [17]:
logreg = LogisticRegression()

In [18]:
logreg = LogisticRegression(C=19)
logreg.fit(X, y)
val_preds = logreg.predict(val_X)
f1_val = f1_score(val_y, val_preds)
f1_val

0.7713144643167216

In [19]:
logreg = LogisticRegression(C=19)

start_time = time.time()
logreg.fit(train_val_X, train_val_y)
learn_time = time.time() - start_time

test_preds = logreg.predict(test_X)
f1_test = f1_score(test_y, test_preds)
f1_test

0.7701947502116852

**LinearSVC**

In [20]:
svc = LinearSVC(C=1)

In [21]:
param = {'C': range (1,20, 2)}

grid = GridSearchCV(svc, param, scoring='f1', cv=4)
start_time = time.time()
grid.fit(X, y)
gridsearch_time = time.time() - start_time

gridsearch_time, grid.best_params_

(106.77343678474426, {'C': 1})

In [22]:
svc.fit(X, y)
val_preds = svc.predict(val_X)
f1_val_SVC = f1_score(val_y, val_preds)
f1_val_SVC

0.7747779132555304

In [23]:
svc = LinearSVC(C=1)

start_time = time.time()
svc.fit(train_val_X, train_val_y)
learn_time_SVC = time.time() - start_time

test_preds = svc.predict(test_X)
f1_test_SVC = f1_score(test_y, test_preds)
f1_test_SVC

0.7788363388907793

**LightGBM**

In [ ]:
lgbm = LGBMClassifier(metric = 'f1_score')

In [25]:
lgbm.fit(X, y, eval_set=(val_X, val_y))
lgbm_val_preds = lgbm.predict(val_X)
f1_val_LGBM = f1_score(val_y, lgbm_val_preds)
f1_val_LGBM

0.7367455512749953

In [26]:
start_time = time.time()
lgbm.fit(train_val_X, train_val_y)
learn_time_LGBM = time.time() - start_time
lgbm_preds = lgbm.predict(test_X)
f1_test_LGBM = f1_score(lgbm_preds, test_y)
f1_test_LGBM

0.7434234234234235

Хотела подтягуть LightGBM при помощи GridSearchCV, но два раза умирало ядро.

## Выводы

Посмотрим на итоговую таблицу по всем моделям.

In [27]:
final = pd.DataFrame(columns = ["F1_score_validation","F1_score_test", 'Learn_time'],
                      index = ["LogisticRegression","LinearSVC", 'LGBMClassifier'])


report.iloc[0] = [f1_val,f1_test, learn_time]
report.iloc[1] = [f1_val_SVC,f1_test_SVC, learn_time_SVC]
report.iloc[2] = [f1_val_LGBM,f1_test_LGBM, learn_time_LGBM]

In [28]:
final

,F1_score_validation,F1_score_test,Learn_time
LogisticRegression,0.771,0.77,19.8
LinearSVC,0.775,0.779,1.29
LGBMClassifier,0.737,0.743,232


В ходе работы над проектом были подготовленны данные обучения на моделях, которые были поделены на обучающую, валидационную и тестовую выборки, обучены модели и выбраны лучшие, которые удовлетворяют поставленной задаче F1>0.75.

Так как TF-IDF превращают текст в численные значения, лучшими моделями стали LogisticRegression и LinearSVC (чуть лучше первой модели). И по метрике и по времени обучения. LGBMClassifier показал себя хуже всех по метрике, а также и по времени обучения модели. 

На тестовой выбоке по метрике F1 аналогично лучше всего себя показали модели LinearSVC и LogisticRegression.  